In [10]:
import numpy as np

def single_mean_squared_error(y, t):
    return 0.5 * np.sum((y-t)**2)

def single_cross_entropy_error(y, t):
    delta = 1e-7
    return -np.sum(t * np.log(y + delta))


In [11]:
t = [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
y1 = [0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0]
y2 = [0.1, 0.05, 0.1, 0.0, 0.05, 0.6, 0.0, 0.1, 0.0, 0.0]
print(single_mean_squared_error(np.array(y1), np.array(t)))
print(single_mean_squared_error(np.array(y2), np.array(t)), '\n')

print(single_cross_entropy_error(np.array(y1), np.array(t)))
print(single_cross_entropy_error(np.array(y2), np.array(t)))

0.09750000000000003
0.5975 

0.510825457099338
2.302584092994546


In [12]:
# 미니배치: 데이터 일부만 사용해서 학습하는 방법
import torchvision

train_dataset = torchvision.datasets.MNIST(
    root='./data',
    train=True,
    download=True
)

x_train = train_dataset.data.numpy()
x_train = x_train.reshape(x_train.shape[0], -1)
t_train = train_dataset.targets.numpy()
t_train = t_train.reshape(t_train.shape[0], -1)

train_size = x_train.shape[0]
batch_size = 10
batch_mask = np.random.choice(train_size, batch_size)
print('batch_mask:', batch_mask)

x_batch = x_train[batch_mask]
t_batch = t_train[batch_mask]
print('x_batch:', x_batch)
print('t_batch:', t_batch)


batch_mask: [16569 48609 12066 18613 15948  5269 55581  8122  4783 42391]
x_batch: [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
t_batch: [[3]
 [6]
 [8]
 [2]
 [2]
 [7]
 [8]
 [3]
 [3]
 [3]]


In [ ]:
# 배치용 손실함수

# 원핫 인코딩
# 어차피 정답 레이블 이외의 y 인덱스는 0으로 처리되기 때문에, label 방식과 결과가 같음
def cross_entropy_error_onehot(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)

    batch_size = y.shape[0]
    return -np.sum(t * np.log(y + 1e-7)) / batch_size

def cross_entropy_error_label(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)

    batch_size = y.shape[0]

    # 정답 레이블이 1차원 배열인 경우
    # if batch_size == 5
    # np.arange(batch_size) = [0, 1, 2, 3, 4]
    # t = [2, 7, 0, 9, 4]
    # y[np.arange(batch_size), t] = [y[0, 2], y[1, 7], y[2, 0], y[3, 9], y[4, 4]]
    # 첫번째 인덱스: 샘플링한 인덱스
    # 두번째 인덱스: 정답 레이블
    return -np.sum(np.log(y[np.arange(batch_size), t] + 1e-7)) / batch_size

## 학습에 정확도를 지표로 하지 않는 이유
매개변수의 미분이 대부분의 장소에서 0이 되기 때문  
정확도는 매개변수의 작은 변화에는 거의 반응을 보이지 않거나, 불연속적으로 갑자기 변화하는 특성이 있음  
(계단함수를 활성화 함수로 사용하지 않는 이유와 같음)